# Importing Libraries

In [1]:
import sqlite3
import pandas as pd

# Connecting to SQL Database

In [6]:
conn = sqlite3.connect('sql-murder-mystery.db')
cur = conn.cursor()

# Finding Missing Crime Scene Report and Joining other tables to find murderer

In [41]:
cur.execute("""
            SELECT *
            FROM crime_scene_report c
            JOIN facebook_event_checkin f
            ON c.date = f.date
            JOIN get_fit_now_member g
            ON g.person_id = f.person_id
            JOIN interview i
            ON i.person_id = g.person_id
            JOIN person p
            ON p.name = g.name
            JOIN income 
            ON p.ssn = income.ssn
            WHERE city = 'SQL City' AND c.date = 20180115 AND description = 'Report Not Found'

            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,date,type,description,city,person_id,event_id,event_name,date,id,person_id,...,person_id,transcript,id,name,license_id,address_number,address_street_name,ssn,ssn,annual_income
0,20180115,assault,Report Not Found,SQL City,67318,4719,The Funky Grooves Tour,20180115,48Z55,67318,...,67318,I was hired by a woman with a lot of money. I ...,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,871539279,10500


# Adding murderer to solution

In [60]:
cur.execute("""
                INSERT INTO solution VALUES (1, "Jeremy Bowers");
""")

# Committing changes to SQL Database

In [61]:
conn.commit()

# Checking Solution

In [63]:
cur.execute("""
                SELECT * 
                FROM solution 
            ;""")
df = pd.DataFrame(cur.fetchall())
df[1]

0    Congrats, you found the murderer! But wait, th...
Name: 1, dtype: object

# Adding suspect that may have hired Jeremy Bowers

In [64]:
cur.execute("""
            SELECT *
            FROM crime_scene_report c
            JOIN facebook_event_checkin f
            ON c.date = f.date
            JOIN get_fit_now_member g
            ON g.person_id = f.person_id
            JOIN interview i
            ON i.person_id = g.person_id
            JOIN person p
            ON p.name = g.name
            WHERE city = 'SQL City' AND c.date = 20180115 AND description = 'Report Not Found'

            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,date,type,description,city,person_id,event_id,event_name,date,id,person_id,...,membership_start_date,membership_status,person_id,transcript,id,name,license_id,address_number,address_street_name,ssn
0,20180115,assault,Report Not Found,SQL City,16371,4719,The Funky Grooves Tour,20180115,90081,16371,...,20160208,gold,16371,"I saw the murder happen, and I recognized the ...",16371,Annabel Miller,490173,103,Franklin Ave,318771143
1,20180115,assault,Report Not Found,SQL City,67318,4719,The Funky Grooves Tour,20180115,48Z55,67318,...,20160101,gold,67318,I was hired by a woman with a lot of money. I ...,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


# Adding Annabel Miller to Solution

In [65]:
cur.execute("""
                INSERT INTO solution VALUES (2, "Annabel Miller");
""")

conn.commit()

cur.execute("""
                SELECT * 
                FROM solution 
            ;""")
df = pd.DataFrame(cur.fetchall())
df[1]

0    Congrats, you found the murderer! But wait, th...
1                                       Annabel Miller
Name: 1, dtype: object

# She is still a suspect :)